# Use TTH_Template Custom Notebook template

In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")

from fosforml.model_manager.snowflakesession import get_session
from fosforml import register_model

# Read data from Snowflake

In [2]:
my_session = get_session()

In [3]:
table_name = 'FLIGHTS'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
flights = df.copy()
flights_needed_data = df.copy()

In [5]:
flights_needed_data

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,2,12,4,WN,697,N399WN,BWI,CLT,840,910.0,30.0,13.0,923.0,95.0,90.0,73.0,361,1036.0,4.0,1015,1040.0,25.0,0,0,None,0.0,0.0,12.0,13.0,0.0
1,2015,2,12,4,WN,523,N232WN,BWI,MCI,840,839.0,-1.0,14.0,853.0,185.0,164.0,146.0,967,1019.0,4.0,1045,1023.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,2015,2,12,4,WN,2513,N8600F,BWI,RSW,840,909.0,29.0,17.0,926.0,175.0,160.0,139.0,919,1145.0,4.0,1135,1149.0,14.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,2015,2,12,4,WN,451,N8647A,BWI,SJU,840,840.0,0.0,36.0,916.0,255.0,260.0,219.0,1565,1355.0,5.0,1355,1400.0,5.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,2015,2,12,4,WN,3497,N441WN,SFO,ATL,840,839.0,-1.0,9.0,848.0,270.0,275.0,261.0,2139,1609.0,5.0,1610,1614.0,4.0,0,0,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,8,26,3,EV,4993,N717EV,ORF,DTW,1430,1425.0,-5.0,10.0,1435.0,114.0,113.0,95.0,529,1610.0,8.0,1624,1618.0,-6.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5819075,2015,8,26,3,EV,4926,N138EV,PHF,ATL,1430,1427.0,-3.0,9.0,1436.0,106.0,101.0,87.0,508,1603.0,5.0,1616,1608.0,-8.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5819076,2015,8,26,3,EV,5092,N582CA,HOU,ATL,1430,1426.0,-4.0,10.0,1436.0,119.0,106.0,91.0,696,1707.0,5.0,1729,1712.0,-17.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5819077,2015,8,26,3,EV,6037,N14570,IAH,AMA,1430,1427.0,-3.0,11.0,1438.0,106.0,103.0,80.0,517,1558.0,12.0,1616,1610.0,-6.0,0,0,None,NaN,NaN,NaN,NaN,NaN


In [6]:
flights_needed_data.value_counts('DIVERTED')  # will tell us the no. of flights which were diverted

DIVERTED
0    5803892
1      15187
Name: count, dtype: int64

In [7]:
# filtering out unnecessary columns
flights_needed_data=flights_needed_data.drop(['YEAR','FLIGHT_NUMBER','AIRLINE__CODE','DISTANCE','TAIL_NUMBER','TAXI_OUT',
                                              'SCHEDULED_TIME','DEPARTURE_TIME','WHEELS_OFF','ELAPSED_TIME',
                                              'AIR_TIME','WHEELS_ON','DAY_OF_WEEK','TAXI_IN','CANCELLATION_REASON'],
                                             axis=1)

In [ ]:
# replacing all NaN values with the mean of the attribute in which they are present
flights_needed_data=flights_needed_data.fillna(flights_needed_data.mean())

In [ ]:
flights_needed_data.head()

In [ ]:
# creating a new column; it will tell if the flight was delayed or not
reslt=[]

In [ ]:
for row in flights_needed_data['ARRIVAL_DELAY']:
  if row > 15:
    result.append(1)
  else:
    result.append(0) 

In [ ]:
flights_needed_data['result'] = result

In [ ]:
flights_needed_data.value_counts('result')

In [ ]:
# removing some more columns
flights_needed_data=flights_needed_data.drop(['ORIGIN_AIRPORT_CODE', 'DESTINATION_AIRPORT_CODE', 'ARRIVAL_TIME', 'ARRIVAL_DELAY'],axis=1)
flights_needed_data

In [ ]:
data = flights_needed_data.values
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)  # splitting in the ratio 70:30

In [ ]:
scaled_features = StandardScaler().fit_transform(X_train, X_test)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [ ]:
pred_prob = clf.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
auc_score